In [ ]:
function count_heads(n)
    c::Int = 0
    for i = 1:n
        c += rand(Bool)
    end
    c
end


In [4]:
workers()

1-element Array{Int64,1}:
 1

In [ ]:
@time count_heads(10_000_000_000)

In [7]:
addprocs(16)

16-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17

In [8]:
workers()

16-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16
 17

In [10]:
@time count_heads(10_000_000_000)

 24.620652 seconds (5 allocations: 176 bytes)


4999882816

In [11]:
@everywhere function count_heads(n)
    c::Int = 0
    for i = 1:n
        c += rand(Bool)
    end
    c
end

In [14]:
a = @spawn count_heads(10_000_000_000)

Future(3, 1, 36, Nullable{Any}())

In [15]:
@time fetch(a)

 21.840213 seconds (8.75 k allocations: 439.473 KiB)


4999992335

In [18]:
b=@spawn count_heads(10_000_000_000)

Future(5, 1, 40, Nullable{Any}())

In [19]:
@time fetch(a)+fetch(b)

 21.745674 seconds (544 allocations: 16.641 KiB)


10000022242

In [20]:
function count_heads2(n)
    c::Int = 0
    @parallel for i = 1:n
        c += rand(Bool)
    end
    c
end

count_heads2 (generic function with 1 method)

In [22]:
@time count_heads2(10_000_000_000)

  0.001661 seconds (2.36 k allocations: 83.188 KiB)


0

Note the addition of the (+) reduction operator below and the effect that it has. 

In [3]:
function count_heads3(n)
    c::Int = 0
    c= @parallel (+) for i = 1:n
        rand(Bool)
    end
    c
end

count_heads3 (generic function with 1 method)

In [4]:
@time count_heads3(10_000_000)

  0.572981 seconds (20.09 M allocations: 310.071 MiB, 4.93% gc time)


4998196

In [29]:
function computePi(size)
    x=rand(size,2)
    count =0;
    for i ∈ 1:size 
        if +(x[i,1]^2,x[i,2]^2) <=1
            count+=1
       # println(count)
        end
    end
 return(4*count/size)
end

computePi (generic function with 1 method)

In [72]:
@time computePi(10000)

  0.000235 seconds (7 allocations: 156.500 KiB)


3.132

In [35]:
workers()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [131]:
@everywhere  f(s,count)=(println("process id = $(myid()) s = $s count = $count");repeat(s,count))

In [132]:
pmap((j1,j2)->f(j1,j2),("x","y","z"),(2,1,3))

	From worker 7:	process id = 7 s = x count = 2
	From worker 5:	process id = 5 s = z count = 3
	From worker 4:	process id = 4 s = y count = 1


3-element Array{String,1}:
 "xx" 
 "y"  
 "zzz"

In [133]:
workers()

8-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8
 9

In [129]:
@everywhere function evalPi(size)
    println("process id = $(myid()) size=$size")
    x=rand(size,2)
    count =0;
    for i ∈ 1:size 
        if +(x[i,1]^2,x[i,2]^2) <=1
            count+=1
       # println(count)
        end
    end
 return(4*count/size)
end

In [135]:
@time pmap((j1)->evalPi(j1),(100,1000,10000,10000,100000))

	From worker 2:	process id = 2 size=10000
	From worker 8:	process id = 8 size=10000
	From worker 6:	process id = 6 size=1000
	From worker 3:	process id = 3 size=100
	From worker 7:	process id = 7 size=100000
  0.458248 seconds (233.73 k allocations: 12.394 MiB, 2.12% gc time)


(3.12, 3.088, 3.1072, 3.1388, 3.1524)

In [126]:
@time computePi(10000)

  0.000200 seconds (7 allocations: 156.500 KiB)


3.162

In [118]:
A = rand(1000,1000);
localMatrix = @spawn A^2;

In [55]:
@time fetch(localMatrix)

  0.000007 seconds (4 allocations: 160 bytes)


1000×1000 Array{Float64,2}:
 260.52   253.76   267.495  264.062  …  259.354  250.77   259.938  264.261
 248.215  247.492  259.034  263.735     249.344  246.069  253.047  257.402
 249.903  246.368  261.854  260.075     251.211  248.078  257.343  253.846
 265.565  252.122  264.351  259.56      258.98   250.352  258.071  262.725
 248.179  247.249  258.766  258.182     253.61   242.182  250.426  254.114
 251.887  237.357  255.759  257.238  …  255.031  239.289  247.214  253.769
 263.012  255.495  266.472  272.497     263.48   248.965  263.786  264.776
 257.413  252.408  264.881  275.088     263.641  258.39   266.159  263.288
 252.401  248.397  258.273  263.821     254.573  242.144  255.405  255.724
 265.012  255.587  262.185  267.713     263.091  254.916  262.622  263.235
 243.642  236.542  247.562  261.098  …  242.117  241.936  245.515  246.527
 260.355  250.617  262.671  261.808     255.072  246.829  262.81   255.918
 264.607  251.277  272.506  267.892     263.93   254.64   262.839  267.1

In [51]:
remoteMatrix = @spawn rand(1000,1000)^2;

In [56]:
@time fetch(remoteMatrix)

  0.000008 seconds (4 allocations: 160 bytes)


1000×1000 Array{Float64,2}:
 247.996  244.262  251.906  253.731  …  245.85   250.348  255.256  249.565
 248.555  239.26   239.544  244.496     244.596  248.454  247.904  245.72 
 242.503  232.143  242.802  240.25      232.577  246.955  245.373  236.031
 252.4    251.932  250.327  251.938     247.719  257.788  262.845  250.929
 248.078  237.56   239.771  241.929     234.168  243.531  242.875  237.163
 244.138  240.365  243.888  241.682  …  244.793  249.125  249.535  241.773
 257.878  245.312  248.935  250.063     249.95   251.193  261.248  247.712
 251.744  242.842  250.96   252.288     251.031  255.031  256.808  251.317
 248.128  238.073  243.132  246.079     242.379  249.644  253.036  239.733
 253.256  243.798  249.896  248.58      249.831  254.489  257.388  249.239
 248.533  241.929  248.439  247.261  …  244.889  250.766  260.006  245.304
 249.856  241.078  253.12   250.999     243.015  249.292  252.808  245.022
 257.295  238.701  252.114  251.329     255.627  255.062  262.93   252.5

In [96]:
@everywhere  f(s,count)=(println("process id = $(myid()) s = $s count = $count");repeat(s,count))

In [102]:
pmap((j1,j2)->f(j1,j2),("x","y","z"),(2,1,3))

	From worker 6:	[1 2 3 4000]process id = 6 s = x count = 2
	From worker 7:	process id = 7 s = z count = 3
	From worker 4:	[1 2000 3 4]process id = 4 s = y count = 1


3-element Array{Void,1}:
 nothing
 nothing
 nothing

In [89]:
@everywhere a = [1 2 3 4]
@parallel  for i=1:4
 a[i]=i*1000
 print(a)
end


4-element Array{Future,1}:
 Future(3, 1, 194, #NULL)
 Future(4, 1, 195, #NULL)
 Future(5, 1, 196, #NULL)
 Future(6, 1, 197, #NULL)

In [91]:
pmap((j1,j2)->f(j1,j2),("a","b","c"),(2,1,3))

	From worker 3:	[1000 2 3 4]process id = 3 s = a count = 2
	From worker 9:	process id = 9 s = b count = 1
	From worker 2:	process id = 2 s = c count = 3


3-element Array{String,1}:
 "aa" 
 "b"  
 "ccc"